Federated learning with Pytorch

In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset  

hyperparamters definition

In [2]:
num_clients = 20 #divide data into different clients 
num_selected = 6 #randomly selected clients 
num_rounds = 150 #number of rounds
epochs = 5 #local training rounds
batch_size = 32 #batch size to load into the data loader

Data loading

In [3]:
train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True,transform=
            torchvision.transforms.Compose(
            [torchvision.transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
# Dividing the training data into num_clients, with each client having equal number of images
traindata_split = torch.utils.data.random_split(train_data, [int(train_data.data.shape[0] / num_clients) for _ in range(num_clients)])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

In [4]:
test_data = torchvision.datasets.MNIST(root='./data', 
            train=False, download=True, 
            transform=torchvision.transforms.
            Compose([torchvision.transforms.ToTensor()]))
test_loader = torch.utils.data.DataLoader(test_data)

In [5]:
image, _ = train_data[0] 
print(image.size())
image_flatten = image.view(image.shape[0], -1)
print (image_flatten.size())
torch.Size([1, 28, 28]) 
torch.Size([1, 784])

torch.Size([1, 28, 28])
torch.Size([1, 784])


torch.Size([1, 784])

We use here a CNN model using Conv2d class: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
model= CNN()

In [9]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data, target
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [10]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [32]:
def test(global_model, test_loader):
    """This function test the global model on test data and returns test loss and test accuracy """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data, target
            output = global_model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

In [33]:
global_model =  model #global model

client_models = [ model for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) # local model sync with global

#optimizer
opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

In [34]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []

#FL training

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    train_losses.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    test_loss, acc = test(global_model, test_loader)
    test_losses.append(test_loss)
    test_acc.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:29<00:00,  4.89s/it]


0-th round
average train loss 0.312 | test loss 0.0918 | test acc: 0.972


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:30<00:00,  5.05s/it]


1-th round
average train loss 0.126 | test loss 0.0725 | test acc: 0.977


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:31<00:00,  5.29s/it]


...

147-th round
average train loss 0.0582 | test loss 0.0421 | test acc: 0.989


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:30<00:00,  5.09s/it]


148-th round
average train loss 0.0736 | test loss 0.0413 | test acc: 0.988


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:31<00:00,  5.21s/it]


149-th round
average train loss 0.0736 | test loss 0.0385 | test acc: 0.989
